# Tailor multiple gates using the integer representation

In  [tutorial.ipynb](./tutorial.ipynb) we learned how to tailor a logical circuit for a stabilizer code. We did this by passing `Connectivity`, `StabilizerCode`, and `Circuit` objects to the `tailor_logical_gate` function. If we want to create circuit implementations for many different logical gates, we may call this function multiple times. If we are looking for a few specific gate implementations, this works fine. On the other hand, one might want to create a large amount of logical gates, say 720 in total; the number of two-qubit Clifford gates modulo Paulis. For this, we can use the function `tailor_multiple_logical_gates`. This function avoids the additional overhead of creating a distinct _Gurobi_ model for all gates, and, more importantly, we can pass a list of integers identifying specific Clifford gates.

In [1]:
import htlogicalgates as htlg

The workflow is similiar to that of `tailor_logical_gate`. The only difference is that, instead of a `Circuit` object, we pass a list of integers.

In [2]:
connectivity = htlg.Connectivity("circular", num_qubits=4)
stab_code = htlg.StabilizerCode("4_2_2")
logical_gates = [0,1,121]
results = htlg.tailor_multiple_logical_gates(stab_code, connectivity, logical_gates, 2)

## Clifford gates represented by integers

For a given number of qubits, every Clifford is identified by an integer. This identification is unique up to Pauli operators and is based on the work _J. Math. Phys. 55, 122202 (2014)_ by Koenig _et. al._ (also available under [arXiv:1406.2170](https://arxiv.org/abs/1406.2170)).

We can work with this representation by using the `Clifford` class. For example, we can turn a `Circuit` object into its corresponding `Clifford` object. Note that all circuits in this package are Clifford circuits.

In [3]:
circ0 = htlg.Circuit("I 1", 2)
circ1 = htlg.Circuit("H 0", 2)
circ2 = htlg.Circuit("H 0\nH 1", 2)
cliff0 = circ0.to_clifford()
cliff1 = circ1.to_clifford()
cliff2 = circ2.to_clifford()
print(f"Clifford integer of cliff0: '{cliff0.clifford_int}'")
print(f"Clifford integer of cliff1: '{cliff1.clifford_int}'")
print(f"Clifford integer of cliff2: '{cliff2.clifford_int}'")

Clifford integer of cliff0: '0'
Clifford integer of cliff1: '1'
Clifford integer of cliff2: '121'


Here, we used the property `clifford_int` to obtain the integer of the Clifford operator modulo Paulis. For completeness, the Pauli part is also identified by an integer.

In [4]:
cliff3 = htlg.Circuit("H 0\nH 1\nX 1").to_clifford()
print(f"Clifford integer of cliff3: '{cliff3.clifford_int}'")
print(f"Pauli integer of cliff3: '{cliff3.pauli_int}'")
print(f"Pauli integer of cliff2: '{cliff2.pauli_int}'")

Clifford integer of cliff3: '121'
Pauli integer of cliff3: '4'
Pauli integer of cliff2: '0'


As shown above, the Clifford integers of both Clifford gates are the same but their Pauli integers differ.

## Using the function `tailor_multiple_gates`

As previously shown, we can use the function `tailor_multiple_gates` in the following manner.

In [5]:
connectivity = htlg.Connectivity("circular", num_qubits=4)
stab_code = htlg.StabilizerCode("4_2_2")
logical_gates = [0, 1, 121]
results = htlg.tailor_multiple_logical_gates(stab_code, connectivity, logical_gates, 2)

The function returns a dictionary containing all tailored gates and meta information.

In [6]:
print(list(results["Meta"].keys()))
print(list(results["Gates"].keys()))

['Connectivity', 'Code', 'Number CZ layers', 'Time limit', 'Started', 'Ended']
[0, 1, 121]


For example, we can query the used connectivity and stabilizer code.

In [7]:
connectivity = results["Meta"]["Connectivity"]
stab_code = results["Meta"]["Code"]

print("Connectivity matrix:\n", connectivity.matrix)
print()
print("Code parameters:\n[[n,k,d]] =",stab_code.nkd)

Connectivity matrix:
 [[0 1 0 1]
 [1 0 1 0]
 [0 1 0 1]
 [1 0 1 0]]

Code parameters:
[[n,k,d]] = (4, 2, 2)


On the other hand, we can query the circuits from the results dictionary by identifying different gates by the integer that represents the logical Clifford action.

In [8]:
for key, val in results["Gates"][121].items():
    print(key, ":", repr(val.__str__()))

Circuit : 'Z 0\nH 0\nZ 1\nX 1\nC_XYZ 1\nZ 2\nX 2\nC_XYZ 2\nH 3\nCZ 0 3\nCZ 1 2\nZ 0\nS 0\nC_XYZ 1\nC_XYZ 2\nZ 3\nS 3\nCZ 1 2\nH 1\nH 2\n'
Status : 'Optimal'
Runtime : '1.937000036239624'


The runtime is given in seconds and the status message is the same as for the function `tailor_logical_gate`.

We can save the dictionary to a .json-file automatically by specifying the `output_file` argument of the function `tailor_multiple_logical_gates`. Alternatively, we can use the functions `save_results_dictionary` to manually save the results to a .json-file.

In [11]:
filepath = "tutorial_results.json"
htlg.save_results_dictionary(results, filepath)

We can load the results dictionary by using the function `load_results_dictionary`.

In [12]:
results_loaded = htlg.load_results_dictionary(filepath) # = results